<a href="https://colab.research.google.com/github/rb58853/images_RIS-ML-Conv-NLP/blob/main/end_model/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Montar drive

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

### Instalar bibliotecas

In [ ]:
!pip install transformers
!pip install sly
!pip install spacy
!python3 -m pip install opencv-python matplotlib
!python3 -m pip install 'git+https://github.com/rb58853/segment-anything.git'

### Importar bibliotecas


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

Si estas usando colab ejecute la siguiente celda para importar imagenes de prueba

In [ ]:
import os
!git clone https://github.com/rb58853/ML-RSI-Images.git
os.chdir('/content/ML-RSI-Images/code/end_model')
#Cargar sam en la direcion nueva
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

### Cargar Imagen
Cargar la imagen que se desea procesar. Para seleccionar una imagen distinta debe cambiar el valor de la variable `img_url`. Si desea cambiar el tamaño minimo que puede tener una segmentacion debe cambiar el valor de la variable `image_partition`.

In [ ]:
images_dir = '../../images/'
img_url = images_dir + 'image_2.jpg'
raw_image = Image.open(img_url).convert("RGB")
image = cv2.imread(img_url)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

image_partition = 64 #tamaño mínimo(en píxeles) de un cuadro de segmentación = tamaño(imagen)/image_partition
weigth, heigth = raw_image.size
area = (weigth * heigth)/image_partition

Seleccione el modelo de blip que desea utilizar, si esta en colab, para correr con el modelo blip 2 necesita una GPU v100 o superior. Se recomienda probar el modelo con `blip_version = 1` si no posee colab pro, o no posee al menos 16GB en su tarjeta grafica.

In [ ]:
blip_version = 1

Para visualizar la imagen cargada ejecutar celda:

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(image)
plt.axis('off')
plt.show()

# Cargar Modelos
Para cargar y ejecutar todos los modelos y funciones necesarias para procesar la imagen ejecutar las celdas que estan comprendidas en este encabezado.


## Segment Anything Model
Modelo de segmentación y funciones adicionales para procesar las salidas y entradas del mismo.


In [ ]:
from features.models import SAM
sam = SAM
sam.import_model()

## BLIP

In [ ]:
from features.models import BLIP, BLIP2
caption_model = BLIP
if blip_version == 2:
    caption_model = BLIP2
caption_model.import_model()    

## CLIP

In [ ]:
from features.models import CLIP
clip = CLIP
clip.import_model()

# Run Model
En estas celdas se ejecutan las funciones creadas y modelos cargados. Si las 3 celdas con un solo click, se mostrara el ranking de las 10 mejores descripciones encontradas y las descripciones de cada una de las imagenes segmentadas junto a su imagen correspondiente:

En esta celda usted puede cambiar los parametros que desee para variar los resultados


In [ ]:
# ['box' or 'mask'] DEFAULT: 'box'
_segmentation = 'box'

# `int` DEFAULT: 80
image_partition = 80
area = (weigth * heigth)/image_partition

# `int` cantidad de descripciones a mostrar DEFAULT: 10
count_rank = 10

# [`True` or `False`] Usar la descripcion reducida DEFAULT: True
use_reduc = False

In [ ]:
segm_images = sam.all_areas_from_image(image, raw_image, min_box_area = area, min_area = area/2)[_segmentation]
captions = caption_model.all_captions_from_list_images(image, segm_images)

In [ ]:
image_caption = str(caption_model.caption(image))
print("image_caption: " + image_caption)
result = clip.select_caption(captions, raw_image)
print("original_caption: "+str(result['caption']))
# rduced_caption = clip.reduce_caption(result['caption'], raw_image)
# print("reduced_caption: "+str(rduced_caption))

# if use_reduc and rduced_caption not in captions:
#     captions.append(rduced_caption)

# result = select_caption(captions, raw_image)

probs = result['probs']
end_captions = clip.short_captions(probs, captions)

long_caption = image_caption
max_long = 2
len_caption_image = len(image_caption)
for caption,_ in zip(end_captions.values(),range(max_long)):
    long_caption += ". " + caption[len_caption_image:]

print("long_caption: "+str(long_caption))
# rduced_long_caption = reduce_caption(long_caption, raw_image)
# print("reduced_long_caption: "+str(rduced_long_caption), end= '\n\n')

print("ranking: ")
end_key = None
last_print = False
for key,value,i in zip(end_captions.keys(),end_captions.values(),range(1000)):
    last_print = True
    if i < count_rank:
        print(str(i+1)+". {:.2f}".format(key * 100) + "%: "+ str(value))
        las_print = False
    end_key = key

if last_print:
    print(".\n.\n.\n"+str(len(end_captions.keys())) + ". {:.2f}".format(end_key * 100) + "%: "+ str(end_captions[end_key]))


#### Tokenizar el caption final (long)

In [ ]:
import tokenize
tokens = tokenize.start(long_caption)
for token in tokens:
    print (token)

# Ver segmentaciones

In [ ]:
show = True
if show:
    for im in segm_images:
        plt.figure(figsize=(2,2))
        plt.title(caption_model.caption(im))
        plt.imshow(im)
        plt.axis('off')
        plt.show()